In [7]:
#https://docs.google.com/spreadsheets/d/1D4H2OoHOFVPmCoyKBVCjxIl0Bt3RLYSz/edit?gid=1727910551#gid=1727910551



import pandas as pd
from datetime import datetime



divChampions = pd.read_excel('U.S.DividendChampions-LIVE.xlsx', sheet_name='All', skiprows=2)
divChampions = divChampions.drop(divChampions.columns[[2, 10, 12, 13, 14, 15, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]], axis=1)



#1: Legalabb 15 eve tarto folyamatos osztalekemeles
rows_to_delete = []

for index, row in divChampions.iterrows():
    if 15 > row.iloc[3]:
        rows_to_delete.append(index)

print("#1. Törölt sorok:\n", divChampions.loc[rows_to_delete].iloc[:, [0, 3]])
divChampions.drop(rows_to_delete, inplace=True)



#2: Az osztalektrendben nem lathato osztalekcsokkenes
#TODO: https://finance.yahoo.com/quote/ADM/history/?filter=div&period1=1563612967&period2=1721465763



#3: Az osztalekemeles uteme (DGR) nem mutat trendszeru romlast.
rows_to_delete.clear()

for index, row in divChampions.iterrows():
    if row.iloc[10] < row.iloc[11] and row.iloc[11] < row.iloc[12] and row.iloc[12] < row.iloc[13]:
        rows_to_delete.append(index)

print("\n#3. Törölt sorok:\n", divChampions.loc[rows_to_delete].iloc[:, [0, 10, 11, 12, 13]])
divChampions.drop(rows_to_delete, inplace=True)

#4: Az indulo osztalekhozam a megfelelo tartomanyban van.
SnPAtlagosOsztalek = 1.32 #be kell helyettesiteni az aktualissal
#consolon bekerni

rows_to_delete.clear()

for index, row in divChampions.iterrows():
    if SnPAtlagosOsztalek * 1.5 > row.iloc[5] or SnPAtlagosOsztalek * 5 < row.iloc[5]:
        rows_to_delete.append(index)

print(f"\n#4. {SnPAtlagosOsztalek * 1.5:.2f} < Div Yield < {SnPAtlagosOsztalek * 5:.2f}\nTörölt sorok:\n", divChampions.loc[rows_to_delete].iloc[:, [0, 5]])
divChampions.drop(rows_to_delete, inplace=True)



#5: Az indulo osztalekhozam es a varhato osztalekemeles osszege > 9%
divChampions['#5 oh+oe'] = divChampions.iloc[:, 5] + divChampions.iloc[:, 10]

rows_to_delete.clear()

for index, row in divChampions.iterrows():
    if row.iloc[15] < 9:
        rows_to_delete.append(index)

print("\n#5. Törölt sorok (<9):\n", divChampions.loc[rows_to_delete].iloc[:, [0, 5, 10, 15]])
divChampions.drop(rows_to_delete, inplace=True)



#6: A (kifizetett osztalek / cash flow) * 100 < 75% es nem mutat trendszeru romlast
#TODO: payout rate a 212-n (a penzugyi ero fulon van meg 1-2 erdekes dolog)



#7: Az osztalékhozam lehetoleg maximalis a multra visszatekintve
#TODO: https://www.profitspi.com/stock/view.aspx?v=stock-chart&uv=101058&p=WTRG#&&vs=638570769637950408
#https://ycharts.com/companies/WTRG/dividend_yield
#https://www.koyfin.com/company/ugi/dividends/

#8
divChampions['#8 Gordon-modell'] = (10 - ((divChampions.iloc[:, 7] * divChampions.iloc[:, 8] / divChampions.iloc[:, 4]) * 100)).round(2)

rows_to_delete.clear()

for index, row in divChampions.iterrows():
    if row.iloc[16] < 3 or row.iloc[16] > 6:
        rows_to_delete.append(index)

print("\n#8. 2 < Gordon-modell < 6\nTörölt sorok:\n", divChampions.loc[rows_to_delete].iloc[:, [0, 16]])
divChampions.drop(rows_to_delete, inplace=True)



current_date = datetime.now().strftime('%Y-%m-%d')
file_name = f'Filtered_DividendChampions_{current_date}.xlsx'
divChampions.to_excel(file_name, index=False)
#esetleg valamilyen szempontok szerint rendezni a reszvenyeket

#1. Törölt sorok:
     Symbol  No Years
0        A        13
1     AAPL        12
2     ABBV        12
4      ABR        12
5      ABT        11
..     ...       ...
724   XRAY        13
725    XYL        13
727    YUM         7
728   ZION        11
729    ZTS        12

[452 rows x 2 columns]

#3. Törölt sorok:
     Symbol  DGR 1Y  DGR 3Y  DGR 5Y  DGR 10Y
21     AIT    2.95    3.04    3.14     4.29
22     AIZ    2.92    3.42    4.35    11.38
25     ALB    1.27    1.29    3.62     5.25
41     AOS    7.02    7.58    9.93    18.16
54    ATRI    6.10    9.65   11.28    13.75
..     ...     ...     ...     ...      ...
688     VZ    1.95    1.99    2.03     2.37
695   WDFC    6.42    7.40    8.98    10.35
708    WMT    1.79    1.82    1.86     1.95
710    WPC   -4.13   -0.85   -0.12     1.84
717    WST    5.48    5.81    6.20     7.18

[84 rows x 5 columns]

#4. 1.98 < Div Yield < 6.60
Törölt sorok:
     Symbol  Div Yield
8      ADI       1.86
37     AMP       1.23
39    ANDE       1.32
53